<a href="https://colab.research.google.com/github/megnaanand/climate-change-analysis/blob/main/climate_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

## 1. Introduction

This data exploration and inference project aims to uncover correlations between environmental changes and a global rise in temperatures over the years 1880 to 2020, utilizing datasets provided by NASA Climate Change.

### 1.1. Project Overview
Briefly describe the project's goals and significance.

### 1.2. Scope of the Project
Define the specific aspects of climate change being studied.

### 1.3. Data Sources
Introduce NASA datasets being used (CO2 and Methane levels, Global Temperature, etc.).


### 1.4. Importance of the Study
Discuss the relevance of understanding climate change indicators.

## 2. Data Collection and Preprocessing
 We will be using  information about global Carbon Dioxide and Methane levels, Global Temperature, Ocean Warming, Ice Sheets Sea Levels, and Arctic Sea Ice levels.


### 2.1. Data Collection
Load the datasets from NASA


In [3]:
df_ice_sheets = pd.read_csv("data/ice_sheets.txt", delim_whitespace=True)
print(df_ice_sheets)


     TIME(year.decimal)  Antarctic-mass(Gigatonnes)  \
0               2002.29                        0.00   
1               2002.35                       18.50   
2               2002.62                      -60.48   
3               2002.71                       44.62   
4               2002.79                       61.76   
..                  ...                         ...   
217             2023.12                    -2460.45   
218             2023.20                    -2362.12   
219             2023.29                    -2151.29   
220             2023.37                    -2297.35   
221             2023.45                    -2298.12   

     Antarctic-mass-1-sigma-uncertainty(Gigatonnes)  
0                                            178.90  
1                                            102.47  
2                                             83.09  
3                                             97.19  
4                                             61.82  
..             

In [4]:
df_c02_levels = pd.read_csv("data/c02_levels.txt", delim_whitespace=True)
print(df_c02_levels)

        # decimal    monthly de-season   #days st.dev   unc.     of
0       #    date    average    alized      of   days    mon   mean
1    1958       3  1958.2027    315.70  314.43     -1  -9.99  -0.99
2    1958       4  1958.2877    317.45  315.16     -1  -9.99  -0.99
3    1958       5  1958.3699    317.51  314.71     -1  -9.99  -0.99
4    1958       6  1958.4548    317.24  315.14     -1  -9.99  -0.99
..    ...     ...        ...       ...     ...    ...    ...    ...
784  2023       6  2023.4583    423.68  421.22     29   0.57   0.20
785  2023       7  2023.5417    421.83  421.45     21   0.48   0.20
786  2023       8  2023.6250    419.68  421.55     21   0.45   0.19
787  2023       9  2023.7083    418.51  421.94     18   0.30   0.14
788  2023      10  2023.7917    418.82  422.17     27   0.44   0.16

[789 rows x 8 columns]


In [5]:
df_global_temperatures = pd.read_csv("data/global_temperatures.txt", delim_whitespace=True)
print(df_global_temperatures)

     Year  No_Smoothing  Lowess(5)
0    1880         -0.17      -0.10
1    1881         -0.09      -0.13
2    1882         -0.11      -0.17
3    1883         -0.18      -0.21
4    1884         -0.29      -0.24
..    ...           ...        ...
138  2018          0.85       0.93
139  2019          0.97       0.92
140  2020          1.01       0.91
141  2021          0.84       0.91
142  2022          0.89       0.90

[143 rows x 3 columns]


In [6]:
df_methane = pd.read_csv("data/methane.txt", delim_whitespace=True)
print(df_methane)

    year     mean   unc
0   1984  1644.85  0.67
1   1985  1657.29  0.59
2   1986  1670.09  0.74
3   1987  1682.70  0.49
4   1988  1693.16  0.67
5   1989  1704.53  0.55
6   1990  1714.43  0.68
7   1991  1724.82  0.72
8   1992  1735.47  0.57
9   1993  1736.50  0.46
10  1994  1742.07  0.58
11  1995  1748.88  0.48
12  1996  1751.28  0.68
13  1997  1754.53  0.42
14  1998  1765.54  0.64
15  1999  1772.34  0.64
16  2000  1773.33  0.81
17  2001  1771.22  0.59
18  2002  1772.66  0.55
19  2003  1777.33  0.50
20  2004  1777.05  0.50
21  2005  1774.16  0.64
22  2006  1774.95  0.56
23  2007  1781.37  0.63
24  2008  1787.01  0.77
25  2009  1793.53  0.69
26  2010  1798.93  0.73
27  2011  1803.14  0.63
28  2012  1808.12  0.63
29  2013  1813.41  0.66
30  2014  1822.57  0.53
31  2015  1834.26  0.55
32  2016  1843.12  0.68
33  2017  1849.58  0.64
34  2018  1857.33  0.82
35  2019  1866.58  0.60
36  2020  1878.93  0.56
37  2021  1895.28  0.55
38  2022  1911.82  0.78


In [7]:
df_sea_levels = pd.read_csv("data/sea_levels.txt", delim_whitespace=True)
print(df_sea_levels)

      altimeter_type  merged_file_cycle_#  year_fraction_of_year  \
0                  0                   11            1993.011526   
1                  0                   12            1993.038692   
2                  0                   13            1993.065858   
3                  0                   14            1993.093025   
4                  0                   15            1993.120191   
...              ...                  ...                    ...   
1119               0                 1131            2023.418439   
1120               0                 1132            2023.445605   
1121               0                 1133            2023.472771   
1122               0                 1134            2023.499937   
1123               0                 1135            2023.527103   

      number_of_observations  number_weighted_observations  GMSL_no_GIA_mm  \
0                     452600                     327401.31          -38.61   
1                     44759

In [8]:
df_ocean_warming = pd.read_json("data/ocean_warming.json", orient='index')

print(df_ocean_warming)

                              0
1992-01-16 12:00:00    0.000000
1992-02-15 12:00:00   -2.448807
1992-03-16 12:00:00   -6.903282
1992-04-16 00:00:00   -7.749611
1992-05-16 12:00:00   -8.800667
...                         ...
2019-08-16 12:00:00  185.437531
2019-09-16 00:00:00  185.978836
2019-10-16 12:00:00  187.566727
2019-11-16 00:00:00  187.734161
2019-12-16 00:00:00  186.917221

[336 rows x 1 columns]


In [11]:
df_arctic_sea_ice = pd.read_excel("data/arctic_sea_ice.xlsx", index_col=0)

print(df_arctic_sea_ice)

      January  February   March   April     May    June    July  August  \
1978      NaN       NaN     NaN     NaN     NaN     NaN     NaN     NaN   
1979   15.414    16.175  16.342  15.447  13.857  12.530  10.311   8.041   
1980   14.862    15.955  16.041  15.429  13.793  12.205  10.100   7.984   
1981   14.910    15.604  15.632  15.010  13.802  12.430  10.271   7.844   
1982   15.177    15.974  16.044  15.466  13.973  12.476  10.367   8.139   
1983   14.942    16.006  16.085  15.172  13.491  12.296  10.570   8.186   
1984   14.473    15.299  15.584  15.015  13.577  12.152   9.977   7.771   
1985   14.725    15.466  15.886  15.357  14.073  12.219   9.744   7.402   
1986   14.890    15.785  15.910  15.059  13.379  11.976  10.146   7.979   
1987   14.970    16.050  15.816  15.211  13.743  12.486  10.333   7.628   
1988      NaN    15.583  15.957  15.123  13.561  11.943   9.809   7.892   
1989   14.955    15.495  15.423  14.328  13.044  12.245  10.130   7.882   
1990   14.783    15.578  

In [12]:
combined_df = pd.concat([df_ice_sheets, df_global_temperatures, df_methane, df_sea_levels, df_ocean_warming, df_arctic_sea_ice], ignore_index=True)
print(combined_df)

       0  Annual  Antarctic-mass(Gigatonnes)  \
0    NaN     NaN                        0.00   
1    NaN     NaN                       18.50   
2    NaN     NaN                      -60.48   
3    NaN     NaN                       44.62   
4    NaN     NaN                       61.76   
...   ..     ...                         ...   
1905 NaN  10.201                         NaN   
1906 NaN  10.150                         NaN   
1907 NaN  10.552                         NaN   
1908 NaN  10.661                         NaN   
1909 NaN  10.393                         NaN   

      Antarctic-mass-1-sigma-uncertainty(Gigatonnes)   April  August  \
0                                             178.90     NaN     NaN   
1                                             102.47     NaN     NaN   
2                                              83.09     NaN     NaN   
3                                              97.19     NaN     NaN   
4                                              61.82     NaN   

### 2.2. Data Preprocessing

#### 2.2.1. Data Cleaning
Handle missing values, outliers, and other inconsistencies

#### 2.2.2. Data Transformation
Normalize, scale, or transform data as needed for analysis.

#### 2.2.3. Data Integration
Combine different datasets into a cohesive format.


## 3. Exploratory Data Analysis (EDA)

### 3.1. Initial Data Exploration
Use basic statistics and visualizations to understand the datasets.

### 3.2. Correlation Analysis

#### 3.2.1. Heatmaps
Visualize correlations between different environmental indicators.

#### 3.2.2. Geographic Heatmaps
Show spatial distribution of climate change indicators.

### 3.3. Time Series Analysis

#### 3.3.1. Line Plots
Examine trends in temperature and other variables over time.

#### 3.3.2. Seasonal Analysis
Investigate seasonal patterns in the data.

## 4. Data Visualization and Inference

### 4.1. Advanced Data Visualization

#### 4.1.1. Interactive Plots
Create dynamic plots for more engaging data exploration.

#### 4.1.2. Comparative Visualizations
Compare different indicators side-by-side

### 4.2. Inference from Visualizations
Draw conclusions from the visualizations created.

## 5. Predictive Modeling

### 5.1. Model Selection
Discuss the selection of predictive models

### 5.2. Model Training and Validation
Train models on historical data and validate their performance.

### 5.3. Forecasting
Use the models to forecast future climate trends based on historical data.

## 6. Discussion and Insights

### 6.1. Key Findings
Summarize the major insights gained from the analysis.

### 6.2. Unexpected Discoveries
Highlight any surprising patterns or results.

### 6.3. Limitations of the Study
Acknowledge any limitations in data or methodology.

## 7. Conclusion and Future Implications

### 7.1. Implications of Findings
Discuss how your findings impact our understanding of climate change.

### 7.2. Recommendations for Future Research
Suggest areas for further investigation.

### 7.3. Final Thoughts
Restate the importance of the project and its contributions to the field.

## 8. References

### 8.1. Data Sources
Cite the NASA datasets and any other sources used.

### 8.2. Additional Literature
Reference any additional literature that supported your analysis

## 9. Appendix

### 9.1. Additional Data
Include any supplementary data or analysis.

### 9.2. Code Documentation
Provide detailed comments and documentation for the code used.